# Image classification: CIFAR dataset

This notebooks experiments with image classification with a different ** model architecture** from those we've been using before

In [3]:
import tensorflow as tf
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
labels  = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

In [5]:
# initialize parameters
batch_size = 64
n_classes = 10
epochs = 100
lr = 1e-4

##### preprocess the dataset

In [ ]:
cfar = tf.keras.datasets.cifar10

(x_train, y_train), (x_test, y_test) = cfar.load_data()

x_train, x_test = x_train / 255., x_test / 255.

In [ ]:
# use one-hot encoding
y_train = tf.one_hot(y_train, depth=10)
y_test = tf.one_hot(y_test, 10)

In [32]:
# create a function to help  display our inputs

def show_images(images, y_labels=None):
    """
        Displays an array of images on a visual plot
    """
    fig=plt.figure(figsize=(8, 8))
    columns = 6
    rows = 4
    for i in range(1, columns*rows +1):
        img = np.reshape(images[i], [w, h])
        ax = fig.add_subplot(rows, columns, i)
        if y_labels is not None:
            ax.set_title(labels[np.argmax(y_labels[i])])


        plt.imshow(img, cmap='Greys')
    plt.tight_layout()
    plt.show()

##### define the model architecture

In [10]:
model = tf.keras.Sequential()

In [12]:
model.add(tf.keras.layers.Conv2D(filters=32,
                                kernel_size=3,
                                padding='same',
                                kernel_regularizer=tf.keras.regularizers.l2(lr),
                                input_shape=(28, 28, 1)))
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.BatchNormalization())

model.add(tf.keras.layers.Conv2D(filters=32,
                                kernel_size=3,
                                padding='same',
                                kernel_regularizer=tf.keras.regularizers.l2(lr)))
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.MaxPool2D())
model.add(tf.keras.layers.Dropout(.2))



model.add(tf.keras.layers.Conv2D(filters=64,
                                kernel_size=3,
                                padding='same',
                                kernel_regularizer=tf.keras.regularizers.l2(lr)))
model.add(tf.keras.layers.Activation('relu'))

model.add(tf.keras.layers.BatchNormalization())

model.add(tf.keras.layers.Conv2D(filters=64,
                                kernel_size=3,
                                padding='same',
                                kernel_regularizer=tf.keras.regularizers.l2(lr))
         )
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.MaxPool2D())
model.add(tf.keras.layers.Dropout(.3))


          
model.add(tf.keras.layers.Conv2D(filters=128,
                                kernel_size=3,
                                padding='same',
                                kernel_regularizer=tf.keras.regularizers.l2(lr)))
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.BatchNormalization())

model.add(tf.keras.layers.Conv2D(filters=128,
                                kernel_size=3,
                                padding='same',
                                kernel_regularizer=tf.keras.regularizers.l2(lr)))
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.MaxPool2D())
model.add(tf.keras.layers.Dropout(.4))

model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(units=n_classes,
                               activation='softmax'))

In [15]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 28, 28, 32)        320       
_________________________________________________________________
activation (Activation)      (None, 28, 28, 32)        0         
_________________________________________________________________
batch_normalization (BatchNo (None, 28, 28, 32)        128       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 28, 28, 32)        9248      
_________________________________________________________________
activation_1 (Activation)    (None, 28, 28, 32)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 28, 28, 32)        128       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 32)        0

In [19]:
model.compile(optimizer=tf.keras.optimizers.Adam(lr),
             loss=tf.keras.losses.categorical_crossentropy,
             metrics=['accuracy'])

#### implement real-time data augmentation

This will apply various **transformations** to the images such as flips, shifts, rotations, which should enable the model **generalize** better

In [24]:
data_gen = tf.keras.preprocessing.image.ImageDataGenerator(
    rotation_range=10,         # randomly rotate images between 0 - 9 degrees
    width_shift_range=0.1,     # shift images horizontally for total width fraction,
    height_shift_range=0.1,    # shift vertically over height fraction
    horizontal_flip=True,
    vertical_flip=True,
    zoom_range=1.0,
    validation_split=.1
)

In [26]:
# setup an early stopping callback

call_back = tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=1e-5, patience=3, 
                                             restore_best_weights=True)

In [ ]:
model.fit_generator(generator=data_gen.flow(x_train, y_train, batch_size=batch_size),
                   epochs=epochs, callbacks=[call_back])

The fit generator method fits the model on the data returned by the `data_gen`

##### Save or load the model

In [ ]:
model.save('.model.cifar.ckpt')

In [ ]:
model.load('.model.cifar.ckpt')

##### Evaluate the model

In [ ]:
model.evaluate(x_test, y_test)

In [ ]:
# show sample prediction
n_samples = 10

y_pred = model.predict(x_test[:n_samples])
show_images(x_test[:n_samples], y_pred)